# Pomegranate quickstart

#### In this document, we apply the loopy belief propagation algorithm of pomegranate to a simple example. 

In [1]:
from pomegranate import *
%load_ext watermark
%watermark -m -n -p pomegranate

Tue Apr 07 2020 

pomegranate 0.12.2

compiler   : Clang 6.0 (clang-600.0.57)
system     : Darwin
release    : 19.0.0
machine    : x86_64
processor  : i386
CPU cores  : 12
interpreter: 64bit


## Defining the probablistic model

### Structure of the model

Our example model is represented by this Bayesian network:

<img src="./images/opengm_quickstart_bn.png" width="200" height="100">

As you know, this graph indicates that the distribution factorizes as follows:

<img src="./images/opengm_quickstart_factorization.png" width = "600" height="200">

To create the Bayesian network in pomegranate, we first create the distributions which live in each node in the graph. For a discrete (aka categorical) bayesian network we use `DiscreteDistribution` objects for the root nodes and `ConditionalProbabilityTable` objects for the inner and leaf nodes. The columns in a `ConditionalProbabilityTable` correspond to the order in which the parents (the second argument) are specified, and the last column is the value the `ConditionalProbabilityTable` itself takes. 

In [2]:
X0 = DiscreteDistribution({'0':0.3, '1':0.7})

X1 = ConditionalProbabilityTable(
    [['0', '0', 0.2], 
     ['0' , '1', 0.8], 
     ['1', '0', 0.7], 
     ['1', '1', 0.3]], [X0])

X2 = ConditionalProbabilityTable(
    [['0', '0', 0.4], 
     ['0' , '1', 0.6], 
     ['1', '0', 0.3], 
     ['1', '1', 0.7]], [X0])

X3 = ConditionalProbabilityTable(
    [['0', '0','0', 0.2], 
     ['0', '0','1', 0.8], 
     ['0', '1','0', 0.3], 
     ['0', '1','1', 0.7], 
     ['1', '0','0', 0.6], 
     ['1', '0','1', 0.4], 
     ['1', '1','0', 0.7], 
     ['1', '1','1', 0.3]], [X1, X2])

Next, we pass these distributions into state objects along with the name for the node.

In [3]:
# State objects hold both the distribution, and a high level name.
s0 = State(X0, name="X0")
s1 = State(X1, name="X1")
s2 = State(X2, name="X2")
s3 = State(X3, name='X3')

Then we add the states to the network. The way the states are added to the network makes no difference to it, and so you should add the states according to how the columns are organized in your data.

In [4]:
# Create the Bayesian network object with a useful name
model = BayesianNetwork("Quick-Start")

# Add the three states to the network 
model.add_states(s0, s1, s2, s3)

Then we need to add edges to the model. The edges represent which states are parents of which other states. This is currently a bit redundant with passing in the distribution objects that are parents for each ConditionalProbabilityTable. Edges are added from parent -> child by calling model.add_edge(parent, child).

In [5]:
# Add edges which represent conditional dependencies, where the second node is 
# conditionally dependent on the first node (Monty is dependent on both guest and prize)
model.add_edge(s0, s1)
model.add_edge(s0, s2)
model.add_edge(s1, s3)
model.add_edge(s2, s3)

Lastly, the model must be baked to finalize the internals. Since Bayesian networks use factor graphs for inference, an explicit factor graph is produced from the Bayesian network during the bake step.

In [6]:
model.bake()

### Performing inference

***model.predict_proba({})***

Bayesian networks are explicitly turned into Factor Graphs when inference is done, wherein the Bayesian network is turned into a bipartite graph with all variables having marginal nodes on one side, and joint tables on the other.

Pomegranate uses the loopy belief propagation algorithm to do inference. This is an approximate algorithm which can yield exact results in tree-like graphs, and in most other cases still yields good results. Inference on a Bayesian network consists of taking in observations for a subset of the variables and using that to infer the values that the other variables take. The most variables which are observed, the closer the inferred values will be to truth. 

We can run inference using the `predict_proba` method and passing in a dictionary of values, where the key is the name of the state and the value is the observed value for that state. If we don't supply any values, we get the marginal of the graph, which is just the frequency of each value for each variable over an infinite number of randomly drawn samples from the graph.

Lets see what happens when we look at the marginal.

In [7]:
model.predict_proba({})

array([{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "0" :0.3000000000000003,
            "1" :0.6999999999999995
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "1" :0.4500000000000003,
            "0" :0.5499999999999997
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "1" :0.6699999999999997,
            "0" :0.3300000000000002
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "1" :0.5529999999999999,
            "0" :0.4470000000000001
        }
    ],
    "frozen" :false
}], dtype=object)

We are returned three `DiscreteDistribution` objects, each representing the marginal distribution for each variable, in the same order they were put into the model. In this case, they represent the guest, prize, and monty variables respectively. We see that everything is equally likely.

We can also pass in an array where `None` (or `np.nan`) correspond to the unobserved values.

In [8]:
model.predict_proba([[None, None, None, None]])

[array([{
     "class" :"Distribution",
     "dtype" :"str",
     "name" :"DiscreteDistribution",
     "parameters" :[
         {
             "0" :0.3000000000000003,
             "1" :0.6999999999999995
         }
     ],
     "frozen" :false
 },
        {
     "class" :"Distribution",
     "dtype" :"str",
     "name" :"DiscreteDistribution",
     "parameters" :[
         {
             "1" :0.4500000000000003,
             "0" :0.5499999999999997
         }
     ],
     "frozen" :false
 },
        {
     "class" :"Distribution",
     "dtype" :"str",
     "name" :"DiscreteDistribution",
     "parameters" :[
         {
             "1" :0.6699999999999997,
             "0" :0.3300000000000002
         }
     ],
     "frozen" :false
 },
        {
     "class" :"Distribution",
     "dtype" :"str",
     "name" :"DiscreteDistribution",
     "parameters" :[
         {
             "1" :0.5529999999999999,
             "0" :0.4470000000000001
         }
     ],
     "frozen" :false
 }], dty

All of the variables that were observed will be the observed value, and all of the variables that were unobserved will be a `DiscreteDistribution` object. This means that `parameters[0]` will return the underlying dictionary used by the distribution.

Let us now set the evidence X3 = 1 and calculate the marginals of other variables subject to this evidence. We do this by passing a dictionary to `predict_proba` with key pairs consisting of the name of the state (in the state object), and the value which that variable has taken, or by passing in a list where the first index is our observation.

In [9]:
model.predict_proba([{"X3":"1"}])

[array([{
     "class" :"Distribution",
     "dtype" :"str",
     "name" :"DiscreteDistribution",
     "parameters" :[
         {
             "0" :0.2270974567747094,
             "1" :0.7729025432252905
         }
     ],
     "frozen" :false
 },
        {
     "class" :"Distribution",
     "dtype" :"str",
     "name" :"DiscreteDistribution",
     "parameters" :[
         {
             "1" :0.27224877455749763,
             "0" :0.7277512254425024
         }
     ],
     "frozen" :false
 },
        {
     "class" :"Distribution",
     "dtype" :"str",
     "name" :"DiscreteDistribution",
     "parameters" :[
         {
             "1" :0.6379925816574398,
             "0" :0.3620074183425603
         }
     ],
     "frozen" :false
 },
        '1'], dtype=object)]

Once the inference step is performed, we can get the marginals:

In [10]:
x = model.predict_proba([{"X3":"1"}])
for i in range(len(x[0])-1):
    print(x[0][i].parameters)

[{'0': 0.2270974567747094, '1': 0.7729025432252905}]
[{'1': 0.27224877455749763, '0': 0.7277512254425024}]
[{'1': 0.6379925816574398, '0': 0.3620074183425603}]


***model.predict()***

This method returns the most likely component for each sample.

In [11]:
model.predict([{}])

[array(['1', '0', '1', '1'], dtype=object)]